In [1]:
#!pip install psycopg2
import mysql.connector
import numpy as np
import psycopg2
import sqlite3

data_dir = 'C:/Users/werne/Downloads/NSQL/'

# Chapter 1 exercise

## SQLITE

#### 1 : Read all the data from bc.csv, how many rows is there ? Columns ? 

In [1]:
conn = sqlite3.connect(data_dir+'bc.db')
c = conn.cursor()

c.execute("pragma table_info(bc);")
ncol = c.fetchall()

c.execute("SELECT COUNT(*) AS row_count FROM bc;")
nrow=c.fetchone()

print("nrow:", nrow[0], "\nncol:", len(ncol))

conn.close()

nrow: 569 
ncol: 33


#### 2 : Insert the bc.csv in the sqlite format.

Done from DB-Browser SQLite

#### 3 : In the same db create a new table called tomatch with two variables id_ and rdm_float. 

In [2]:
conn = sqlite3.connect('/home/werner/Téléchargements/mydatabase.db')
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS tomatch;")

c.execute("""
    CREATE TABLE tomatch (
        id_ INTEGER,
        rdm_float REAL);
""")

#### 4 : In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.

In [3]:
ids = [int(id) for id in np.arange(50000, 10000000, 2)]
random_floats = np.random.random(size=len(ids))
data_to_insert = list(zip(ids, random_floats))

conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.executemany("INSERT INTO tomatch (id_, rdm_float) VALUES (?, ?);", data_to_insert)
conn.commit()

c.execute("SELECT * FROM tomatch LIMIT 10;")
print(c.fetchall())
conn.close()

[(50000, 0.2787700069315615), (50002, 0.10952689337426691), (50004, 0.012772532612756904), (50006, 0.01611287562926933), (50008, 0.20033476399007155), (50010, 0.11889506556783913), (50012, 0.7154868614034331), (50014, 0.5277685553107473), (50016, 0.952092012050516), (50018, 0.42848353730169086)]


#### 5 : Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

In [5]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
    SELECT AVG(bc.area_mean) AS total_area, MAX(bc.area_mean) AS max_area
    FROM bc
    JOIN tomatch ON bc.id = tomatch.id_;
""")
aggregate_results = c.fetchone()
conn.close()

print("Average area_mean:", aggregate_results[0], "Max Area Mean:", aggregate_results[1])

Average area_mean: 639.783950617284 Max Area Mean: 2250.0


#### 6 : How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

In [6]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
    SELECT COUNT(*)
    FROM bc
    WHERE radius_mean > 15;
""")
radius15p = c.fetchone()

c.execute("""
    SELECT COUNT(*)
    FROM bc
    WHERE radius_mean > 15 and texture_mean > 20;
""")
rad15p_tex20p = c.fetchone()

print("radius > 15 :", radius15p[0],"\nradius > 15 & texture > 20 :", rad15p_tex20p[0])
conn.close()

radius > 15 : 173 
radius > 15 & texture > 20 : 106


#### 7 : For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups

In [12]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
    SELECT diagnosis, ROUND(AVG(perimeter_mean), 2)
    FROM bc
    GROUP BY diagnosis;
""")
mean_results = c.fetchall()
mean_results

difference = abs(mean_results[0][1] - mean_results[1][1])

print(f"Difference between the means of both groups : {difference:.2f}")

Difference between the means of both groups : 37.29


#### 8 : Change the name of the column "diagnosis" to "label"

In [13]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
CREATE TABLE bc_new AS SELECT diagnosis AS label, * FROM bc;
""")
c.execute("""
DROP TABLE bc;
""")
c.execute("""
ALTER TABLE bc_new RENAME TO bc;
""")
c.execute("""
ALTER TABLE bc
DROP COLUMN diagnosis;
""")

#### 9 : Add a column named "day" of type DATE.

In [16]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
ALTER TABLE bc
ADD day date;
""")

#### 10 : Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)

In [19]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
ALTER TABLE bc
ADD COLUMN area_mean_int INTEGER;
""")
c.execute("""
UPDATE bc
SET area_mean_int = CAST(ROUND(area_mean) AS INTEGER);
""")
conn.commit()

#### 11 : create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.

In [20]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS tomatch2;")

c.execute("""
    CREATE TABLE tomatch2 (
        id_ INTEGER,
        rdm_float REAL);
""")

ids = np.arange(50000, 10000001, 10)
random_floats = np.random.random(size=len(ids))

ids = [int(id) for id in ids]

data_to_insert = list(zip(ids, random_floats))

c.executemany("INSERT INTO tomatch2 (id_, rdm_float) VALUES (?, ?);", data_to_insert)
conn.commit()

c.execute("SELECT * FROM tomatch2 LIMIT 10;")
print(c.fetchall())

conn.close()

[(50000, 0.9869485738002751), (50010, 0.7149528371732383), (50020, 0.015280501392798018), (50030, 0.04970064260280349), (50040, 0.04236412628881647), (50050, 0.8486819720537664), (50060, 0.5647073042374567), (50070, 0.881317422843903), (50080, 0.6474744871610636), (50090, 0.519967676215781)]


#### 12 : Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2. 

In [25]:
conn = sqlite3.connect(data_dir+'mydatabase.db')
c = conn.cursor()

c.execute("""
    SELECT *
    FROM bc
    INNER JOIN tomatch ON bc.id = tomatch.id_ AND bc.radius_mean > 15 AND tomatch.rdm_float > 0.50
    INNER JOIN tomatch2 ON bc.id = tomatch2.id_ AND bc.radius_mean > 15 AND tomatch2.rdm_float > 0.50
    LIMIT 10;
""")

result = c.fetchone()
result

('M',
 879830,
 17.01,
 20.26,
 109.7,
 904.3,
 0.08772,
 0.07304,
 0.0695,
 0.0539,
 0.2026,
 0.05223,
 0.5858,
 0.8554,
 4.106,
 68.46,
 0.005038,
 0.01503,
 0.01946,
 0.01123,
 0.02294,
 0.002581,
 19.8,
 25.05,
 130.0,
 1210.0,
 0.1111,
 0.1486,
 0.1932,
 0.1096,
 0.3275,
 0.06469,
 None,
 None,
 904,
 879830,
 0.9598127618556695,
 879830,
 0.8702652490649188)

## MySQL

#### 1 : How many orders were placed in total?

In [1]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="sample8s"
)

c = mydb.cursor()

c.execute( """
    SELECT COUNT(*)
    FROM sample_superstore
""")

count = c.fetchone()
print(count[0], 'Orders')

350 Orders


#### 2 : What is the total sales amount?

In [2]:
c = mydb.cursor()

c.execute( """
    SELECT SUM(Sales)
    FROM sample_superstore
""")

count = c.fetchone()
print("Total sales :", round(count[0],2),"$")

Total sales : 261110.62 $


#### 3 : What is the average profit per order?

In [3]:
c = mydb.cursor()

c.execute( """
    SELECT AVG(Profit)
    FROM sample_superstore
""")

count = c.fetchone()
print(round(count[0],2), '$ average profit')

136.04 $ average profit


#### 4 : Which customer has placed the most orders?

In [4]:
c = mydb.cursor()

c.execute( """
    SELECT COUNT(*), `Customer Name`
    FROM sample_superstore
    GROUP BY `Customer Name`
    ORDER BY COUNT(*) DESC
    LIMIT 1;
""")

count = c.fetchone()
print(count[1],"placed", count[0], "orders")

Tony Wilkins Winters placed 24 orders


#### 5 : What is the total profit for each product category?

In [5]:
c = mydb.cursor()

c.execute( """
    SELECT `Product Category`, SUM(Profit)
    FROM sample_superstore
    GROUP BY `Product Category`
""")

count = c.fetchall()
print(count)

[('Office Supplies', 23211.967080000002), ('Technology', 20450.68018400001), ('Furniture', 3952.1451930000067)]


#### 6 : What is the average discount percentage?

In [6]:
c = mydb.cursor()

c.execute( """
    SELECT AVG(Discount)
    FROM sample_superstore
""")

count = c.fetchone()
print(round(count[0]*100,2),"%")

4.97 %


#### 7 : Which region has the highest total sales?

In [7]:
c = mydb.cursor()

c.execute( """
    SELECT Region, SUM(Sales) 
    FROM sample_superstore
    GROUP BY Region
    ORDER BY SUM(Sales) DESC
    LIMIT 1;
""")

count = c.fetchone()
print("Highest total sales :", round(count[1]), "$ in region", count[0])

Highest total sales : 151357 $ in region West


#### 8 : How many orders were shipped using each shipping mode?

In [8]:
c = mydb.cursor()

c.execute( """
    SELECT `Ship Mode`, COUNT(*) 
    FROM sample_superstore
    GROUP BY `Ship Mode`;
""")

count = c.fetchall()

for i in range(len(count)):
    print(count[i][0],":",count[i][1],"orders")

Regular Air : 262 orders
Express Air : 46 orders
Delivery Truck : 42 orders


#### 9 : What is the total quantity ordered for each product sub-category?

In [9]:
c = mydb.cursor()

c.execute( """
    SELECT `Product Sub-Category`, SUM(`Quantity ordered new`) 
    FROM sample_superstore
    GROUP BY `Product Sub-Category`;
""")

count = c.fetchall()
for i in range(len(count)):
    print(count[i][0],":", count[i][1],"orders")

Labels : 342 orders
Pens & Art Supplies : 392 orders
Paper : 572 orders
Scissors, Rulers and Trimmers : 48 orders
Telephones and Communication : 504 orders
Office Machines : 246 orders
Chairs & Chairmats : 332 orders
Appliances : 368 orders
Bookcases : 78 orders
Tables : 100 orders
Envelopes : 92 orders
Office Furnishings : 466 orders
Rubber Bands : 238 orders
Binders and Binder Accessories : 258 orders
Storage & Organization : 242 orders
Copiers and Fax : 10 orders
Computer Peripherals : 138 orders


#### 10 : Which product has the highest profit margin?

In [10]:
c = mydb.cursor()

c.execute( """
    SELECT DISTINCT(`Product Name`), `Product Base Margin`
    FROM sample_superstore
    ORDER BY `Product Base Margin` DESC
    LIMIT 3;
""")

count = c.fetchall()
for i in range(len(count)):
    print(count[i][0],":", count[i][1],"Base margin")

Accessory29 : 0.83 Base margin
IrisÂ® 3-Drawer Stacking Bin, Black : 0.83 Base margin
Accessory37 : 0.81 Base margin


#### 11 : What is the total sales amount for each year?

In [11]:
c = mydb.cursor()

c.execute( """
    SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) AS Year, SUM(Sales)
    FROM sample_superstore
    GROUP BY YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y'))
    ORDER BY Year ASC;
""")

count = c.fetchall()
for i  in range(len(count)):
    print(count[i][0],":", round(count[i][1],2),"$ total sales")

2010 : 50755.56 $ total sales
2011 : 24096.24 $ total sales
2012 : 89786.42 $ total sales
2013 : 96472.4 $ total sales


#### 12 : Which customer segment has the highest average sales?

In [13]:
c = mydb.cursor()

c.execute( """
    SELECT AVG(Sales), `Customer Segment`
    FROM sample_superstore
    GROUP BY `Customer Segment`
    ORDER BY AVG(Sales) DESC
    LIMIT 1;
""")

count = c.fetchall()
print(count[0][1],":", round(count[0][0],2),"$ average sales")

Small Business : 1041.56 $ average sales


## PostgreSQL

In [ ]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

table_name = "bank_full"
columns = [
    "age INTEGER",
    "job VARCHAR(100)",
    "marital VARCHAR(20)",
    "education VARCHAR(100)",
    "default_ VARCHAR(10)",
    "balance NUMERIC",
    "housing VARCHAR(10)",
    "loan VARCHAR(10)",
    "contact VARCHAR(100)",
    "day INTEGER",
    "month VARCHAR(20)",
    "duration INTEGER",
    "campaign INTEGER",
    "pdays INTEGER",
    "previous INTEGER",
    "poutcome VARCHAR(100)",
    "y VARCHAR(10)"
]

create_table_query = (
    "CREATE TABLE IF NOT EXISTS " + table_name + " (" +
    ", ".join(columns) +
    ")"
)

cur.execute(create_table_query)

conn.commit()
cur.close()
conn.close()

#### 1 : What is the average age of the customers?

In [133]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT AVG(age)
    FROM bank_full;
""")
avg = cur.fetchone()

print(round(avg[0],0),"years old in average")

41 years old in average


#### 2 : How many customers have defaulted on loans?

In [134]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT COUNT(*)
    FROM bank_full
    WHERE default_ ='yes';
""")
custom_cnt = cur.fetchone()

print(round(custom_cnt[0],0),"customers defaulted on loans")

815 customers defaulted on loans


#### 3 : What is the maximum balance in the account?

In [136]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT MAX(balance)
    FROM bank_full;
""")
max = cur.fetchone()

print(round(max[0],0),"is the maximum account balance")

102127 is the maximum account balance


#### 4 : How many customers have a housing loan?

In [137]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT COUNT(*)
    FROM bank_full
    WHERE loan='yes';
""")
loan_nb = cur.fetchone()

print(round(loan_nb[0],0),"customers have a housing loan")

7244 customers have a housing loan


#### 5 : What is the education level of the youngest customer?

In [140]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT education, age
    FROM bank_full
    ORDER BY age ASC
    LIMIT 1;
""")
educ = cur.fetchone()

print("education level :", educ[0])

education level : primary


#### 6 : What is the marital status of the oldest customer?

In [141]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT marital, age
    FROM bank_full
    ORDER BY age DESC
    LIMIT 1;
""")
mary = cur.fetchone()

print(mary[0])

divorced


#### 7 : What is the distribution of the months in which customers were last contacted?

In [142]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )

cur = conn.cursor()

cur.execute("""
    SELECT month, COUNT(*)
    FROM bank_full
    GROUP BY month;
""")
months = cur.fetchall()

for i in range(len(months)):
    print(months[i][1], "customers were last contacted in",months[i][0])

738 customers were last contacted in oct
2932 customers were last contacted in apr
6895 customers were last contacted in jul
13766 customers were last contacted in may
1403 customers were last contacted in jan
3970 customers were last contacted in nov
214 customers were last contacted in dec
6247 customers were last contacted in aug
5341 customers were last contacted in jun
477 customers were last contacted in mar
579 customers were last contacted in sep
2649 customers were last contacted in feb


#### 8 : What is the average duration of contact for customers who have a personal loan?

In [147]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT AVG(duration)
    FROM bank_full
    WHERE loan='yes';
""")
dura = cur.fetchone()

print("average duration :", round(dura[0]))

average duration : 251


#### 9 : How many customers were previously contacted, and what was the average duration of their last contact?

In [149]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT COUNT(*), AVG(duration)
    FROM bank_full
    WHERE previous!=0;
""")
contacts = cur.fetchone()

print(contacts[0],"previously contacted customers", "\nAverage duration of their last contact :", round(contacts[1]))

8257 previously contacted customers 
Average duration of their last contact : 260


#### 10 : What is the distribution of job types among customers?

In [150]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT job, COUNT(*)
    FROM bank_full
    GROUP BY job
    ORDER BY COUNT(*) DESC;
""")
jobs = cur.fetchall()

for i in range(len(jobs)):
    print(jobs[i][0],":",jobs[i][1])

blue-collar : 9732
management : 9458
technician : 7597
admin. : 5171
services : 4154
retired : 2264
self-employed : 1579
entrepreneur : 1487
unemployed : 1303
housemaid : 1240
student : 938
unknown : 288


#### 11 : How many customers have been contacted more than once in the current campaign?

In [151]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT COUNT(*)
    FROM bank_full
    WHERE campaign > 1;
""")
contacted = cur.fetchone()

print(contacted[0],"customers contacted more than once")

27667 customers contacted more than once


#### 12 : What is the success rate of previous marketing campaigns (poutcome) among customers

In [152]:
conn = psycopg2.connect(dbname="mydb", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

cur.execute("""
    SELECT COUNT(*)
    FROM bank_full
    WHERE poutcome='success';
""")
success = cur.fetchone()


cur = conn.cursor()
cur.execute("""
    SELECT COUNT(*)
    FROM bank_full;
""")
total = cur.fetchone()

print("success rate :", round(100*success[0]/total[0],2),"%")

success rate : 3.34 %
